Buidling a web scraper for amazon products

In [2]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

Getting the url for multiple pages 

In [ ]:
def get_url(search_text):
    """Generate a url from search term"""
    template = 'https://www.amazon.co.uk/s?k={}&crid=WE27RFFCTW2R&sprefix=pink+%2Caps%2C397&ref=nb_sb_noss_2'
    search_term = search_text.replace(' ', '+')
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page={}'
    
    return url

Extracting desired data

In [ ]:
def extract_record(item):
    """Extract and return data from single record"""
    
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.co.uk' + atag.get('href')

    #price
    try:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    #price before
    try:
        price_before = item.find('span','a-price a-text-price').text
    except AttributeError:
        return
    
    #rating & review count
    try: 
        rating = item.i.text
        review_count = item.find('span','a-size-base s-underline-text').text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, price_before, rating, review_count, url)
    
    return result

Running scraper to open Amazon

In [ ]:
def main(search_term):
    """"run main program routine"""
    #start up web driver
    driver_service = Service(executable_path='C:/Users/kodai/Documents/Coding/chromedriver_win32/chromedriver.exe')
    driver = webdriver.Chrome(service=driver_service)
    
    records = []
    url = get_url(search_term)
    
    for page in range(1,8):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)

    driver.close()
    


Saving to a file

In [ ]:
    #save data to csv
    with open('#Enter file name.csv','w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Review', 'Review_Count', 'URL'])
        writer.writerows(records)


In [ ]:
main('#Enter search keyword')